# Scraping automatizado

En esta última lección vamos a programar un script que sea capaz de scrapear una página web de citas automáticamente, y no, no me refiero a una web de citas para conocer otras personas sino citas de diferentes autores, lo que en inglés se denomina *quote*.

Se trata de una página preparada con fines educativos: https://quotes.toscrape.com/, dejo también [un enlace al archivo](https://web.archive.org/web/20220712030814/https://quotes.toscrape.com/) por si queda inaccesible.

La web tiene diferentes páginas donde aparecen las citas célebres, con su texto, autor y unos tags de categoría. Nos permite buscar en el índice global página a página o directamente por etiquetas:

![](docs/img01.png)

## Requisitos

El programa que vamos a crear constará de una clase `Citas` que recuperará todas las citas de la web y tendrá cuatro métodos estáticos:

* `scrapear()`: Realizará el scrapeo de las citas en todas las páginas de la web.
* `lista(limite)`: Imprimirá las primeras N citas de la lista, podemos cambiar el limite.
* `etiqueta(nombre)`: Imprimirá las citas con una etiqueta concreta.
* `autor(nombre)`: Imprimirá las citas de un autor concreto.

Ejemplos de uso:

```python
Citas.scrapear()                # Scrapear todas las citas de la web
Citas.lista()                   # Imprimir las primeras 10 citas (por defecto)
Citas.lista(20)                 # Imprimir las primeras 20 citas
Citas.etiqueta("love")          # Citas con etiqueta 'love'
Citas.autor("Albert Einstein")  # Citas del autor 'Albert Einstein'
```

Si queréis os lo podéis tomar como un reto, aunque no es la finalidad de la lección, os dejo un par de consejos:

* En la parte inferior hay un botón llamado *Next* para ir pasando a la siguiente página, podemos usarlo para iterar las páginas dinámicamente.
* Scapear una vez es mejor que scrapear dos veces, en ese sentido puede ser muy útil almacenar el contenido en un fichero para ahorrarnos múltiples peticiones web y el tiempo que eso conlleva.

¡Vamos a por ello!

## Pruebas de desarrollo

Empecemos por lo más esencial, dada la portada de la página veamos si podemos extraer las citas con su respectivo autor y etiquetas.

Si inspeccionamos la estructura de cada cita, se basa en una capa `div` con la clase `quote`, dentro un `span` con clase `text` contiene el texto, un tag `small` con clase `author` el autor y dentro de otra `div` con clase `tags` tenemos diferentes los tags en enlaces `a` con la clase `tag`:

In [222]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://quotes.toscrape.com")
soup = BeautifulSoup(req.text)

# Buscamos las citas de la portada
quotes_tags = soup.select("div.quote")
for quote_tag in quotes_tags:
    # Buscamos el texto
    print(quote_tag.select("span.text")[0].getText())
    # Buscamos el autor
    print(quote_tag.select("small.author")[0].getText())
    # Buscamos las etiquetas
    for tag in quote_tag.select("div.tags a.tag"):
        print(tag.getText(), end=" ")
    # Salto de línea para separar las citas
    print("\n")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 



Bien, ya tenemos por donde empezar, podríamos adaptar este código a una función que a partir de una porción de la URL almacene mediante diccionarios las citas:

In [223]:
def scrap_quotes(url=""):
    domain = "https://quotes.toscrape.com"
    req = requests.get(f"{domain}{url}")
    soup = BeautifulSoup(req.text)

    # Lista para almacenar diccionarios que contendrán datos de las citas
    quotes = []
    # Buscamos las citas de la portada
    quotes_tags = soup.select("div.quote")
    for quote_tag in quotes_tags:
        # Creamos un diccionario vacío
        quote = {}
        # Almacenamos los diferentes campos en el diccinario
        quote['text'] = quote_tag.select("span.text")[0].getText()
        quote['author'] = quote_tag.select("small.author")[0].getText()
        quote['tags'] = []
        for tag in quote_tag.select("div.tags a.tag"):
            quote['tags'].append(tag.getText())
        # Añadimos el diccionario con la cita a la lista
        quotes.append(quote)
    # Devolvemos las citas scrapeadas
    return quotes

quotes = scrap_quotes()

for quote in quotes:
    print(quote["text"])
    print(quote["author"])
    for tag in quote["tags"]:
        print(tag, end=" ")
    print("\n")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 



La clave es utilizar nuestra función de forma recursiva detectando si la página tiene el enlace **Next** y cargando la siguiente página de manera que podamos. Veamos cómo extraer el enlace con la siguiente página si la hay:

In [224]:
req = requests.get(domain)
soup = BeautifulSoup(req.text)

# Buscamos el enlace en el tag li con clase next
link_tag = soup.select("li.next a")
# Si hay como mínimo un enlace extraemos su href relativo sumado al dominio
if len(link_tag) > 0:
    next_url = link_tag[0]['href']
    print(next_url)

/page/2/


Podemos integrar este código en nuestra función `scrap_quotes` para devolver no solo las citas de la página, sino también si hay una página siguiente:

In [225]:
def scrap_quotes(url=""):
    domain = "https://quotes.toscrape.com"
    req = requests.get(f"{domain}{url}")
    soup = BeautifulSoup(req.text)

    # Lista para almacenar diccionarios que contendrán datos de las citas
    quotes = []
    # Buscamos las citas de la portada
    quotes_tags = soup.select("div.quote")
    for quote_tag in quotes_tags:
        # Creamos un diccionario vacío
        quote = {}
        # Almacenamos los diferentes campos en el diccinario
        quote['text'] = quote_tag.select("span.text")[0].getText()
        quote['author'] = quote_tag.select("small.author")[0].getText()
        quote['tags'] = []
        for tag in quote_tag.select("div.tags a.tag"):
            quote['tags'].append(tag.getText())
        # Añadimos el diccionario con la cita a la lista
        quotes.append(quote)
        
    # Buscamos el enlace en el tag li con clase next
    next_url = None
    link_tag = soup.select("li.next a")
    # Si hay como mínimo un enlace extraemos su href relativo sumado al dominio
    if len(link_tag) > 0:
        next_url = link_tag[0]['href']
    
    # Imprimiros un mensaje informativo
    print(f"Página {domain}{url}, {len(quotes)} citas scrapeadas.")
    
    # Devolvemos las citas scrapeadas y la siguiente página, que puede ser None
    return quotes, next_url

quotes, next_url = scrap_quotes()

print() # Espacio en blanco
print(next_url)

Página https://quotes.toscrape.com, 10 citas scrapeadas.

/page/2/


Ahora se viene la parte interesante, vamos a implementar una función que scrapee todas las páginas mientras haya una siguente o, alternativamente, podemos establecer un límite para optimizar el proceso y no saturar al servidor:

In [226]:
def scrap_site(limit=2):
    # Definimos una lista global para almacenar todas las citas
    all_quotes = []
    # Definimos la siguiente URL que irá cambiando (inicialmente es el dominio raíz)
    next_url = "" 
    # Iniciamos un bucle infinito
    while 1:
        # Scrapeamos la página, guardamos las citas scrapeadas y la siguiente página
        quotes, next_url = scrap_quotes(next_url)
        # Añadimos las citas scrapeadas a la lista global
        all_quotes += quotes
        # Restamos 1 al limite 
        limit -= 1
        # Si lo superamos o no hay siguiente página finalizamos la función
        if limit == 0 or next_url == None:
            # Finalizamos la función
            return all_quotes

quotes = scrap_site()

print() # Espacio en blanco
for quote in quotes:
    print(quote["text"])
    print(quote["author"])
    for tag in quote["tags"]:
        print(tag, end=" ")
    print("\n")

Página https://quotes.toscrape.com, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/2/, 10 citas scrapeadas.

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood

Ahí la tenemos, una función capaz de scrapear todas las citas de la página por defecto limitado a 2 páginas.

## Implementando la clase Citas

Vamos a ponernos con la clase `Citas` y el método `scrapear` pero siguiendo el consejo que os dí de crear un fichero donde almacenar todas las citas.

### Guardado en fichero

Solo generaremos el fichero si ejecutamos el método `scrapear`, los demás métodos `lista`, `etiqueta` y `autor` analizarán el contenido del fichero volcado en la memoria, pero nunca scrapearán nada directamente.

Después de valorarlo he decidido utilizar un CSV. Lo único que nos dará algún problema es guardar una lista como un campo del registro, pero podemos recuperarla evaluándola de nuevo, ya veréis:

In [227]:
import csv

class Citas:
    
    # Variable de clase para almacenar las citas en la memoria
    quotes = []
    
    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Citas.quotes = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("quotes.csv", "w") as file:
            # Definimos el objeto para escribir con las cabeceras de los campos 
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            # Escribimos las cabeceras
            writer.writeheader()
            # Escribimos cada cita en la memoria en el fichero
            for quote in Citas.quotes:
                writer.writerow(quote)
            
Citas.scrapear()

Página https://quotes.toscrape.com, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/2/, 10 citas scrapeadas.


En este punto deberíamos tener un fichero `quotes.json` con todas las citas, lo que podríamos hacer es cargar en la memoria todas las citas del fichero en caso de que éste exista. De paso podemos implementar el método `lista` para consultarlas:

In [228]:
import os
import csv

class Citas:
    
    # Variable de clase para almacenar las citas en la memoria
    quotes = []
    
    # Recuperamos las citas en la memoria si existe el fichero quotes.json
    if os.path.exists("quotes.csv"):
        with open("quotes.csv", "r") as file:
            data = csv.DictReader(file)
            for quote in data:
                # La lista es una cadena, hay que reevaluarla
                quote['tags'] = eval(quote['tags'])
                quotes.append(quote)
    
    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Citas.quotes = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("quotes.csv", "w") as file:
            # Definimos el objeto para escribir con las cabeceras de los campos 
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            # Escribimos las cabeceras
            writer.writeheader()
            # Escribimos cada cita en la memoria en el fichero
            for quote in Citas.quotes:
                writer.writerow(quote)
            
    @staticmethod
    def listar(limite=10):
        for quote in Citas.quotes[:limite]:
            print(quote["text"])
            print(quote["author"])
            for tag in quote["tags"]:
                print(tag, end=" ")
            print("\n")
            
Citas.listar(5)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 



### Filtro por etiqueta y autor

Ya solo nos falta implementar los métodos de filtrado por etiqueta y autor, es muy fácil porque solo tenemos que recorrer las citas y comprobar si concuerdan con los valores que pasamos a los métodos:

In [229]:
import os
import json

class Citas:
    
    # Variable de clase para almacenar las citas en la memoria
    quotes = []
    
    # Recuperamos las citas en la memoria si existe el fichero quotes.json
    if os.path.exists("quotes.csv"):
        with open("quotes.csv", "r") as file:
            data = csv.DictReader(file)
            for quote in data:
                # La lista es una cadena, hay que reevaluarla
                quote['tags'] = eval(quote['tags'])
                quotes.append(quote)
    
    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Citas.quotes = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("quotes.csv", "w") as file:
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            writer.writeheader()
            for quote in Citas.quotes:
                writer.writerow(quote)
            
    @staticmethod
    def listar(limite=10):
        for quote in Citas.quotes[:limite]:
            print(quote["text"])
            print(quote["author"])
            for tag in quote["tags"]:
                print(tag, end=" ")
            print("\n")

    @staticmethod
    def etiqueta(nombre=""):
        for quote in Citas.quotes:
            if nombre in quote["tags"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")
                
    @staticmethod
    def autor(nombre=""):
        for quote in Citas.quotes:
            if nombre == quote["author"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")

Veamos cuantas citas tenemos con el tag **love**:

In [230]:
Citas.etiqueta("love")

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 

“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never find that half who makes you whole and that goes for everything. Just because you fail once, doesn't mean you're gonna fail at everything. Keep trying, hold on, and always, always, always believe in yourself, beca

Y del autor **Albert Einstein**:

In [231]:
Citas.autor("Albert Einstein")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“If you can't explain it to a six year old, you don't understand it yourself.”
Albert Einstein
simplicity understand 



## Scrapeo de la web completa

El programa está limitado a las 2 primeras páginas, voy a reescribir el código con un límite muy grande que garantice un scrapeo completo de la web:

In [232]:
import os
import csv
import requests
from bs4 import BeautifulSoup


def scrap_quotes(url=""):
    domain = "https://quotes.toscrape.com"
    req = requests.get(f"{domain}{url}")
    soup = BeautifulSoup(req.text)
    
    quotes = []
    quotes_tags = soup.select("div.quote")
    for quote_tag in quotes_tags:
        quote = {}
        quote['text'] = quote_tag.select("span.text")[0].getText()
        quote['author'] = quote_tag.select("small.author")[0].getText()
        quote['tags'] = []
        for tag in quote_tag.select("div.tags a.tag"):
            quote['tags'].append(tag.getText())
        quotes.append(quote)
        
    next_url = None
    link_tag = soup.select("li.next a")
    if len(link_tag) > 0:
        next_url = link_tag[0]['href']
        
    print(f"Página {domain}{url}, {len(quotes)} citas scrapeadas.")
        
    return quotes, next_url


def scrap_site(limit=2):
    all_quotes = []
    next_url = "" 
    while 1:
        quotes, next_url = scrap_quotes(next_url)
        all_quotes += quotes
        limit -= 1
        if limit == 0 or next_url == None:
            return all_quotes

        
class Citas:
    quotes = []
    
    if os.path.exists("quotes.csv"):
        with open("quotes.csv", "r") as file:
            data = csv.DictReader(file)
            for quote in data:
                quote['tags'] = eval(quote['tags'])
                quotes.append(quote)
    
    @staticmethod
    def scrapear():
        Citas.quotes = scrap_site(limit=99) # <--- LIMITE MUY GRANDE
        with open("quotes.csv", "w") as file:
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            writer.writeheader()
            for quote in Citas.quotes:
                writer.writerow(quote)
            
    @staticmethod
    def listar(limite=10):
        for quote in Citas.quotes[:limite]:
            print(quote["text"])
            print(quote["author"])
            for tag in quote["tags"]:
                print(tag, end=" ")
            print("\n")

    @staticmethod
    def etiqueta(nombre=""):
        for quote in Citas.quotes:
            if nombre in quote["tags"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")
                
    @staticmethod
    def autor(nombre=""):
        for quote in Citas.quotes:
            if nombre == quote["author"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")

Vamos a ejecutar el scrapeo completo:

In [233]:
Citas.scrapear()

Página https://quotes.toscrape.com, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/2/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/3/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/4/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/5/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/6/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/7/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/8/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/9/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/10/, 10 citas scrapeadas.


Veamos cuantas citas encuentra ahora con el tag **love**:

In [234]:
Citas.etiqueta("love")

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 

“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never find that half who makes you whole and that goes for everything. Just because you fail once, doesn't mean you're gonna fail at everything. Keep trying, hold on, and always, always, always believe in yourself, beca

Y cuantas del autor **Albert Einstein**:

In [235]:
Citas.autor("Albert Einstein")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“If you can't explain it to a six year old, you don't understand it yourself.”
Albert Einstein
simplicity understand 

“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.”
Albert Einstein
children fairy-tales 

“Logic will get you from A to Z; imagination will get you everywhere.”
Albert Einstein
imagination 

“Any fool can know. The point is to understand.”
Albert Einstein
knowledge learning understanding wisdom 

“Life is like riding a b

Parece que todo funciona correctamente y podemos hacer tantas consultas como queramos sin repetir una y otra vez el proceso de scrapeo. En la práctica podríamos configurar un script que scrapee la página una vez al día para tener el fichero CSV sincronizado.

En cualquier caso con esto acabamos este ejemplo y también la sección, espero que hayáis aprendido mucho.